In [98]:
import pandas as pd
import numpy as np

# Exercise: Exploratory Data Analysis with Pandas
## Instructions
<!-- 1. Data Collection
    - Search and collect data on the following: (1993 - 2023)
        - Israel interest tares
        - US interest rates
        - USD to ILS exchange rates
    - For each dataset, create a pandas DataFrame with the following columns:
        - Year
        - Month
        - Current_Rate (Interest rate or exchange rate for that month)
        - Change_From_Last_Month (Difference in Current_Rate from the previous month)
        - Optional: You may include additional columns if you find them useful (e.g., Change_Percentage, Date).
 -->


In [122]:
# Creating DataFrame
us_interest_rates = pd.read_csv("./exem/FEDFUNDS.csv")

us_interest_rates["Year"] = pd.to_datetime(us_interest_rates["DATE"]).dt.year
us_interest_rates["Month"] = pd.to_datetime(us_interest_rates["DATE"]).dt.month
us_interest_rates = us_interest_rates.rename(columns={"FEDFUNDS": "Current_Rate"})
us_interest_rates["Change_From_Last_Month"] = us_interest_rates["Current_Rate"].diff()
us_interest_rates = us_interest_rates[["Year", "Month", "Current_Rate", "Change_From_Last_Month"]]

Israeli_interest_rate = pd.read_csv("./exem/BR.csv")
Israeli_interest_rate["Year"] = pd.to_datetime(Israeli_interest_rate["TIME_PERIOD"]).dt.year
Israeli_interest_rate["Month"] = pd.to_datetime(Israeli_interest_rate["TIME_PERIOD"]).dt.month
Israeli_interest_rate["Day"] = pd.to_datetime(Israeli_interest_rate["TIME_PERIOD"]).dt.day
Israeli_interest_rate = Israeli_interest_rate.rename(columns={"OBS_VALUE": "Current_Rate"})
Israeli_interest_rate = Israeli_interest_rate.loc[Israeli_interest_rate.groupby(["Year", "Month"])["Day"].idxmin()]

Israeli_interest_rate["Change_From_Last_Month"] = Israeli_interest_rate["Current_Rate"].diff()
Israeli_interest_rate = Israeli_interest_rate[["Year", "Month", "Current_Rate", "Change_From_Last_Month"]]
Israeli_interest_rate

conversion_ratios = pd.read_csv("./exem/USD_ILS - נתונים היסטוריים (1).csv")
conversion_ratios.columns = conversion_ratios.columns.str.lower()
conversion_ratios["Year"] = pd.to_datetime(conversion_ratios["תאריך"], format="%d.%m.%Y", dayfirst=True).dt.year
conversion_ratios["Month"] = pd.to_datetime(conversion_ratios["תאריך"], format="%d.%m.%Y", dayfirst=True).dt.month
conversion_ratios["Day"] = pd.to_datetime(conversion_ratios["תאריך"], format="%d.%m.%Y", dayfirst=True).dt.day
conversion_ratios = conversion_ratios.rename(columns={"שער": "Current_Rate"})
conversion_ratios = conversion_ratios.loc[conversion_ratios.groupby(["Year", "Month"])["Day"].idxmin()]


conversion_ratios["Change_From_Last_Month"] = conversion_ratios["Current_Rate"].diff()
conversion_ratios = conversion_ratios[["Year", "Month", "Current_Rate", "Change_From_Last_Month"]]


conversion_ratios


,Year,Month,Current_Rate,Change_From_Last_Month
2815,2012,3,3.7500,NaN
2804,2012,4,3.7252,-0.0248
2783,2012,5,3.7774,0.0522
2760,2012,6,3.9091,0.1317
2739,2012,7,3.9234,0.0143
...,...,...,...,...
109,2022,8,3.3680,-0.1557
86,2022,9,3.3786,0.0106
64,2022,10,3.5363,0.1577
43,2022,11,3.5313,-0.0050


In [ ]:
# your code here

## Part A: Single DataFrame Analysis 
1. Display the first 5 rows of the US interest rates DataFrame.

2. Calculate the average Israeli interest rate over the period covered by your data.

3. Find the month and year when the USD to ILS exchange rate had the highest increase from the previous month.

4. Determine the total number of months where the US interest rate decreased compared to the previous month.

5. Add a new column to the Israeli interest rates DataFrame called Change_Percentage, representing the percentage change from the previous month.



In [38]:
# A - 1
# your code here
us_interest_rates.head(5)

,Year,Month,Current_Rate,Change_From_Last_Month
0,1993,1,3.02,NaN
1,1993,2,3.03,0.01
2,1993,3,3.07,0.04
3,1993,4,2.96,-0.11
4,1993,5,3.00,0.04


In [126]:
# A - 2
# your code here
Israeli_interest_rate["Current_Rate"].mean()

4.894722222222222

In [127]:
# A - 3
# your code here
big_diffrent_month = conversion_ratios.loc[conversion_ratios["Change_From_Last_Month"].idxmax()]

yearly_change = conversion_ratios.groupby("Year")["Change_From_Last_Month"].sum().reset_index()
yearly_change = yearly_change.rename(columns={"Change_From_Last_Month": "Change_From_Last_Year"})

big_diffrent_year =  yearly_change.loc[yearly_change["Change_From_Last_Year"].idxmax()]

big_diffrent_month

Year                      2022.0000
Month                        7.0000
Current_Rate                 3.5237
Change_From_Last_Month       0.1805
Name: 130, dtype: float64

In [81]:
# A - 4
# your code here

decrease_months = us_interest_rates[us_interest_rates["Change_From_Last_Month"] < 0]
decrease_months.shape[0]

123

In [85]:
# A - 5
# your code here

Israeli_interest_rate['Change_Percentage'] = Israeli_interest_rate['Current_Rate'].pct_change() * 100
Israeli_interest_rate.head(5)

,Year,Month,Current_Rate,Change_From_Last_Month,Change_Percentage
4,1994,1,10.5,NaN,NaN
32,1994,2,10.5,0.0,0.00000
63,1994,3,10.5,0.0,0.00000
93,1994,4,10.5,0.0,0.00000
124,1994,5,11.5,1.0,9.52381


<!-- ## Part B: Merging DataFrames
1. Merge the US and Israeli interest rates DataFrames on Year and Month.

2. In the merged DataFrame, create a new column Rate_Difference that shows the difference between the Israeli and US interest rates for each month.

3. Identify all months when the Israeli interest rate was at least 1% higher than the US interest rate - IH (Israeli Higher).
Identify all months when the US interest rate was at least 1% higher than the Israeli interest rate - UH (US Higher).
Identify all months when the interest rates were within 1% of each other - N (Near)
make a new column called Rate_Comparison with the following values: IH, UH, or N.

4. Merge the exchange rate DataFrame with the merged interest rates DataFrame.

5. After merging all three DataFrames, find the correlation between the Rate_Difference and the Current_Rate of the exchange rate.
hint: find the connection between the difference in interest rates and the exchange rates, use any method you think is appropriate.
provide an explanation of the results (in Hebrew) - at least 2 rows of text + a proof of the connection(some output) you found.
 -->

In [92]:
# B - 1

merged_df = pd.merge(us_interest_rates, Israeli_interest_rate, on=['Year', 'Month'], suffixes=('_US','_Israel'))

merged_df


,Year,Month,Current_Rate_US,Change_From_Last_Month_US,Current_Rate_Israel,Change_From_Last_Month_Israel,Change_Percentage
0,1994,1,3.05,0.09,10.50,NaN,NaN
1,1994,2,3.25,0.20,10.50,0.0,0.00000
2,1994,3,3.34,0.09,10.50,0.0,0.00000
3,1994,4,3.56,0.22,10.50,0.0,0.00000
4,1994,5,4.01,0.45,11.50,1.0,9.52381
...,...,...,...,...,...,...,...
353,2023,6,5.08,0.02,4.75,0.0,0.00000
354,2023,7,5.12,0.04,4.75,0.0,0.00000
355,2023,8,5.33,0.21,4.75,0.0,0.00000
356,2023,9,5.33,0.00,4.75,0.0,0.00000


In [96]:
# B - 2
# your code here
merged_df["Rate_Difference"] = merged_df['Current_Rate_US'] - merged_df['Current_Rate_Israel']
merged_df

,Year,Month,Current_Rate_US,Change_From_Last_Month_US,Current_Rate_Israel,Change_From_Last_Month_Israel,Change_Percentage,Rate_Difference
0,1994,1,3.05,0.09,10.50,NaN,NaN,-7.45
1,1994,2,3.25,0.20,10.50,0.0,0.00000,-7.25
2,1994,3,3.34,0.09,10.50,0.0,0.00000,-7.16
3,1994,4,3.56,0.22,10.50,0.0,0.00000,-6.94
4,1994,5,4.01,0.45,11.50,1.0,9.52381,-7.49
...,...,...,...,...,...,...,...,...
353,2023,6,5.08,0.02,4.75,0.0,0.00000,0.33
354,2023,7,5.12,0.04,4.75,0.0,0.00000,0.37
355,2023,8,5.33,0.21,4.75,0.0,0.00000,0.58
356,2023,9,5.33,0.00,4.75,0.0,0.00000,0.58


In [104]:
# B - 3

merged_df['Rate_Comparison'] = merged_df.apply(
    lambda row: 'IH' if row['Current_Rate_Israel'] >= row['Current_Rate_US'] * 1.01 else
                ('UH' if row['Current_Rate_US'] >= row['Current_Rate_Israel'] * 1.01 else 'N'),
    axis=1
)
merged_df

,Year,Month,Current_Rate_US,Change_From_Last_Month_US,Current_Rate_Israel,Change_From_Last_Month_Israel,Change_Percentage,Rate_Difference,Rate_Comparison
0,1994,1,3.05,0.09,10.50,NaN,NaN,-7.45,IH
1,1994,2,3.25,0.20,10.50,0.0,0.00000,-7.25,IH
2,1994,3,3.34,0.09,10.50,0.0,0.00000,-7.16,IH
3,1994,4,3.56,0.22,10.50,0.0,0.00000,-6.94,IH
4,1994,5,4.01,0.45,11.50,1.0,9.52381,-7.49,IH
...,...,...,...,...,...,...,...,...,...
353,2023,6,5.08,0.02,4.75,0.0,0.00000,0.33,UH
354,2023,7,5.12,0.04,4.75,0.0,0.00000,0.37,UH
355,2023,8,5.33,0.21,4.75,0.0,0.00000,0.58,UH
356,2023,9,5.33,0.00,4.75,0.0,0.00000,0.58,UH


In [107]:
# B - 4
# your code here

all_marged =  pd.merge(merged_df, conversion_ratios, on=['Year', 'Month'])
all_marged

,Year,Month,Current_Rate_US,Change_From_Last_Month_US,Current_Rate_Israel,Change_From_Last_Month_Israel,Change_Percentage,Rate_Difference,Rate_Comparison,Current_Rate,Change_From_Last_Month
0,2012,3,0.13,0.03,2.50,0.00,0.000000,-2.37,IH,3.7099,NaN
1,2012,4,0.14,0.01,2.50,0.00,0.000000,-2.36,IH,3.7613,0.0514
2,2012,5,0.16,0.02,2.50,0.00,0.000000,-2.34,IH,3.9042,0.1429
3,2012,6,0.16,0.00,2.25,-0.25,-10.000000,-2.09,IH,3.9081,0.0039
4,2012,7,0.16,0.00,2.25,0.00,0.000000,-2.09,IH,3.9841,0.0760
...,...,...,...,...,...,...,...,...,...,...,...
125,2022,8,2.33,0.65,2.00,0.75,60.000000,0.33,UH,3.3283,-0.0685
126,2022,9,2.56,0.23,2.00,0.00,0.000000,0.56,UH,3.5628,0.2345
127,2022,10,3.08,0.52,2.75,0.75,37.500000,0.33,UH,3.5214,-0.0414
128,2022,11,3.78,0.70,3.25,0.50,18.181818,0.53,UH,3.4076,-0.1138


In [125]:
# B - 5
# your code here

all_marged[["Rate_Difference","Current_Rate"]].corr()["Current_Rate"]["Rate_Difference"]



-0.1856860276461831

## Part C: Analyzing Relationships Between DataFrames
1. Analyze whether increases in US interest rates are associated with increases or decreases in the USD to ILS exchange rate in the same month.

2. Determine if changes in the Israeli interest rate have a stronger impact on the exchange rate than changes in the US interest rate.

3. Find any time lags in the effect of interest rate changes on the exchange rate. For example, does a change in US interest rates affect the exchange rate in the following month?
(כלומר- האם כשיש שינוי בריבית כלשהי - האם מייד יש שינוי בשער ההמרה? או שיש דיליי של חודש/חודשים עד שמרגישים את ההשפעה - אם בכלל?)

4. Calculate the rolling past 3-month average of the exchange rate for each data time, and add it as a new column.

5. Identify any periods where both US and Israeli interest rates were decreasing, and analyze how the exchange rate behaved during those periods.



In [ ]:
# pert - c 



# Bonus
choose 3 global / local events or periods that you think could have affected the exchange rate or interest rates
and analyze the effect of these events on the data you collected.
show proof of your analysis that supports your claim.

(כלומר- תבחנו אירועים שקרו בתקופה שאתם עובדים עליה ובדקו איך הם השפיעו על הנתונים שאתם עובדים איתם)
למשל: בחירות בארה״ב, בחירות בישראל, רוסיה אוקראינה, מלחמות בישראל וכו׳.

In [ ]:
# Your code here